In [1]:
import numpy as np
import pandas as pd
import sys

# TO DO
# implement: substituion(), consider_Strategy()
# fix guards for team abbreviations in command line call
# make sure team abbreviations match for both of the 2 df's
# update file path for 2 df's once we move code to a server on cs
# actually adjust the fatigue adjustment function in the loop of the main program at the bottom
# possibly adjust probability of scoring or time between scores if game scores are unrealistic
# implement a "baseline" strategy to compete against our agent and lose


class State():
    quarter = 1
    time_in_qtr = 12.00
    home_team = [] # needs to be a df probably with their: minutes in game, fatigue-adjusted-plusminus, "real plus/minus" & maybe Names
    away_team = []
    home_players_on = [] # this can just be a list of the 5 players who the coach has put in the game... maybe use their names for convenience
    away_players_on = [] 
    home_strategy = "Normal"
    away_strategy = "Normal"
    home_score = 0
    away_score = 0
    momentum = 0 # lets say positive means home has scored last, negative means away scored last
    momentum_queue = 0 # so we can track maybe the last 5 baskets or so... rather than just streak
    possession_of_ball = "Neither" # or "Away" or "Home"
    home_timeouts_left = 6 # is this correct for nba? i think
    away_timeouts_left = 6

def consider_timeout(team, state):
    probability_of_calling_timeout = 0.50 # we can totally arbitrarily redefine this, so that it makes sense such that teams don't...
    # ... waste all their timeouts early and so that they dont save them all up too long... but it's mainly based on momentum
    if state.possession_of_ball == team:
        if team == "Home":
              if state.home_timeouts_left > 0:
                    if state.momentum < -5:
                        # away team has momentum
                        u = np.random.uniform()
                        if u > probability_of_calling_timeout:
                            return True
    elif team == "Away":
        if state.away_timeouts_left > 0:
            if state.momentum > 5:
              # home team has momentum
                u = np.random.uniform()
                if u > probability_of_calling_timeout:
                    return True
    return False

In [6]:
def donothing():
  x = 0
  # literally do nothing


def substitution(team, state):
  if team == "Home":
    # do stuff
  elif team == "Away":
    # do stuff


def consider_strategy(team, state):
  write function here


def simulate_points(state):
  # account for who has ball, FAPM of both teams, time in game, strategies, momentum
  home_fapm = sum(state.home_players_on['FAPM'])
  away_fapm = sum(state.away_players_on['FAPM'])
  if(state.possession_of_ball == "Home"):
    u = np.random.uniform()
    if(u > 0.50 + home_fapm - away_fapm):
      # 0 points - turnover
      donothing()
    else:
      u = np.random.uniform()
      if(u < 2/3):
        state.home_score += 2
      elif(2/3 < u < 0.3+(2/3)):
        state.home_score += 3
      else:
        state.home_score += 1
    state.possession_of_ball = "Away"
  elif(state.possession_of_ball == "Away"):
    u = np.random.uniform()
    if(u > 0.50 + away_fapm - home_fapm):
      # 0 points - turnover
      donothing()
    else:
      u = np.random.uniform()
      if(u < 2/3):
        state.away_score += 2
      elif(2/3 < u < 0.3+(2/3)):
        state.away_score += 3
      else:
        state.away_score += 1
    state.possession_of_ball = "Home"

def simulate_time(state, home_avg, away_avg):
    if(state.possession_of_ball == "Home"):
    u = np.random.exponential(1/home_avg)
    elif(state.possession_of_ball == "Away"):
    u = np.random.exponential(1/away_avg)
    # for all players on court, update minutes played
    state.home_players_on['Minutes in Game'].add(u) # for every row in the column
    state.away_players_on['Minutes in Game'].add(u) # for every row in the column
    if(state.time_in_qtr - u < 0):
        state.quarter += 1
        state.time_in_qtr = 12.00
        if(first_poss == "Home"):
            if(state.quarter%2 == 0):
                state.possession_of_ball = "Away"
            else:
                state.possession_of_ball = "Home"
        elif(first_poss == "Away"):
            if(state.quarter%2 != 0):
                state.possession_of_ball = "Away"
            else:
                state.possession_of_ball = "Home"
        return False # false means do not simulate points
    else:
        state.time_in_qtr -= u
        return True # means do proceed to simulate points

#### begin main program ####

home_tm = sys.argv[1]
away_tm = sys.argv[2]
list_of_tms = ["ATL", "BOS", "BKN", "CHA", "CHI", "CLE", "DAL", "DEN","DET", "GSW", "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP",
               "NYK", "OKC", "ORL", "PHI", "PHX", "POR", "SAS", "SAC", "TOR", "UTA", "WAS"]

if(home_tm == "UTAH"):
    home_tm = "UTA" # if the name is close to somehting, we should fix it instead of crashing, for home and away
if(home_tm == "GS"):
    home_tm = "GSW"
if(home_tm == "NO"):
    home_tm = "NOP"
if(home_tm == "NY"):
    home_tm = "NYK"
if(home_tm == "SA"):
    home_tm = "SAS"
if(home_tm == "WSH"):
    home_tm = "WAS"
 # if the name is close to somehting, we should fix it instead of crashing, for home and away
if(away_tm == "UTAH"):
    away_tm = "UTA"
if(away_tm == "GS"):
    away_tm = "GSW"
if(away_tm == "NO"):
    away_tm = "NOP"
if(away_tm == "NY"):
    away_tm = "NYK"
if(away_tm == "SA"):
    away_tm = "SAS"
if(away_tm == "WSH"):
    away_tm = "WAS"
if(home_tm not in list_of_tms):
    print("first team is not a valid team")
    raise Error
if(away_tm not in list_of_tms):
    print("second team is not a valid team")
    raise Error
    
state = State()
all_players = pd.read_csv("NBA_RPM.csv")
temp = all_players[all_players['TEAM'] == home_tm].sort_values(by=['MPG'], ascending=False)[0:10]

'''
for(rown, row in enumerate(temp.itertuples())){
    newRow = {}
    newRow['Name'] = row.NAME
    newRow['FAPM'] = row.RPM
    newRow['RPM'] = row.RPM
    newRow['Minutes in Game'] = 0
    state.home_team.append(newRow)
}
state.home_team = pd.DataFrame(state.home_team)
temp = all_players[all_players['TEAM'] == away_tm].sort_values(by=['MPG'], ascending=False)[0:10]
for(rown, row in enumerate(temp.itertuples())){
    newRow = {}
    newRow['Name'] = row.NAME
    newRow['FAPM'] = row.RPM
    newRow['RPM'] = row.RPM
    newRow['Minutes in Game'] = 0
    state.away_team.append(newRow)
}
'''
state.away_team = pd.DataFrame(state.away_team)
poss_times = pd.read_csv("NBA_Poss.csv")
home_tm_poss_secs = poss_times[poss_times['TEAM'] == home_tm]['SPP']
away_tm_poss_secs = poss_times[poss_times['TEAM'] == away_tm]['SPP']
# initialization
u = np.random.uniform()
if(u < 0.5):
      state.possession_of_ball = "Home"
      first_poss = "Home"
else:
      state.possession_of_ball = "Away"
      first_poss = "Away"


## begin simulation
while(state.quarter < 5):
    consider_timeout("Home", state)
    consider_timeout("Away", state)
    substitution("Home", state)
    substitution("Away", state)
    consider_strategy("Home", state)
    consider_strategy("Away", state)
    try_score = simulate_time(state, home_tm_poss_secs, away_tm_poss_secs)
    if(try_score):
    simulate_points(state)
    # update FAPM and minutes in full roster
    # for each player in home_team, if they are in home_players_on, home_team[player][minutes] = home_players_on[player][minutes]
    # and same for away
    for r1 in range(5):
        for r2 in range(state.home_tm.shape[0]):
            if(state.home_tm['Name'][r2] == state.home_players_on['Name'][r1]):
                state.home_tm['Minutes in Game'][r2] = state.home_players_on['Minutes in Game'][r1]
        for r2 in range(state.away_tm.shape[0]):
            if(state.away_tm['Name'][r2] == state.away_players_on['Name'][r1]):
                state.away_tm['Minutes in Game'][r2] = state.away_players_on['Minutes in Game'][r1]
    #state.home_team["tmp"] = np.where((state.home_team["Name"] == state.home_players_on["Name"]) , state.home_players_on['Minutes in Game'], np.nan)
    #state.home_team[state.home_team["tmp"] != np.nan]['Minutes in Game'] = state.home_team["tmp"][state.home_team["tmp"] != np.nan]
    # one last thing in the loop
    # then update everyones FAPM based on minutes played... we can start with something linear like FAPM = RPM - minutesplayed//4 (integer division by 4)
    for r2 in range(state.home_tm.shape[0]):
        state.home_tm['FAPM'][r2] = state.home_tm['RPM'][r2] - state.home_tm['Minutes in Game'][r2]//4
    for r2 in range(state.away_tm.shape[0]):
        state.away_tm['FAPM'][r2] = state.away_tm['RPM'][r2] - state.away_tm['Minutes in Game'][r2]//4






IndentationError: expected an indented block (1072529669.py, line 36)

In [50]:
state = State()

all_players = pd.read_csv("NBA_RPM.csv")
home_tm = 'ATL'
temp = all_players[all_players['TEAM'] == home_tm].sort_values(by=['MPG'], ascending=False)[0:10]

home = {}
home['NAME'] = temp['NAME'] 
home['RPM'] = temp['RPM'] 
home['Minutes in Game'] = 0
home['FAPM'] = temp['RPM']

state.home_team = []
state.home_team.append(home)
state.home_team = pd.DataFrame.from_dict(state.home_team[0], orient='columns')

away_tm = 'BOS'
temp = all_players[all_players['TEAM'] == away_tm].sort_values(by=['MPG'], ascending=False)[0:10]

away = {}
away['NAME'] = temp['NAME'] 
away['RPM'] = temp['RPM'] 
away['Minutes in Game'] = 0
away['FAPM'] = temp['RPM']

state.away_team = []
state.away_team.append(away)
state.away_team = pd.DataFrame.from_dict(state.away_team[0], orient='columns')
